In [1]:
from cmath import nan
from math import isnan
import difflib
from tkinter.tix import COLUMN 
import pandas as pd
import numpy as np
import json
from pathlib import Path
import lib.pdfield as pdfield
#from matplotlib import pyplot as plt


In [2]:
%reload_ext autoreload
%autoreload 2

### Setup connection to Field-database-server based on config.json file ###
config = pdfield.loadconfigs('config\config.json')
api = pdfield.couchDB_APIs(config)

In [3]:
### Import all documents from Target Field-database ###
targetDOCs= pdfield.getAllDocs(api)
targetDF, docfields  = pdfield.allDocsToDf(targetDOCs)


The database uruk contains 26909 docs.
This is round 1offset : 0
This is round 2offset : 10000
This is round 3offset : 20000
26909
['created', 'modified', '_id', '_rev', 'project', 'resource.identifier']


In [4]:
filtertargetDF = targetDF.apply(pdfield.docsByCreationdate, startdate='2023-03-06', axis=1)
targetDF = targetDF[filtertargetDF]
print(targetDF)

                     identifier  \
1      AUWE1drawing_Tafel86_167   
3                   BaM17_p80_7   
9                BaM19_p422_172   
10             AUWE1_Tafel83_14   
11                BaM19_p461_30   
...                         ...   
26896               U23_Layer16   
26900  AUWE1drawing_Tafel42_351   
26903  AUWE1drawing_Tafel32_181   
26905            BaM23_p500_422   
26906  AUWE1drawing_Tafel37_nan   

                                               relations  \
1      {'depicts': ['fd4d75e2-1f97-42bc-9cc3-ff2c64c2...   
3      {'depicts': ['7add1cb7-1ab7-4710-8fd6-d9650c27...   
9      {'liesWithin': ['cad691d1-cd2f-4d1a-b329-3c1fd...   
10     {'liesWithin': ['e3199988-127c-4dca-8abe-c6999...   
11     {'liesWithin': ['cad691d1-cd2f-4d1a-b329-3c1fd...   
...                                                  ...   
26896  {'isRecordedIn': ['621f1722-39df-4acc-be7e-cdd...   
26900  {'depicts': ['5a901314-f608-4755-a1a1-acedfd16...   
26903  {'depicts': ['59b6e129-014c-461b

In [15]:
%reload_ext autoreload
%autoreload 2
samples = targetDF[targetDF['type']=='Sample']
print(samples)
samplesCeramics= pdfield.filterDFBySubstringInField(samples, 'description', 'ceram')
#print(samplesCeramics['identifier'])
samplesPottery= pdfield.filterDFBySubstringInField(samples, 'description', 'potter')
currentPottery = targetDF[targetDF['type']=='Pottery']
#currentPottery = currentPottery[~currentPottery['description'].str.contains('not exported')]
#print(currentPottery)
#currentMollusc = targetDF[targetDF['type']=='Mollusc']
###currentMollusc['sampleType']='molluscs'
#print(currentMollusc['identifier'])
#print(currentPottery)
exportPottery =  pd.concat([currentPottery ,samplesPottery , samplesCeramics], ignore_index=True)
#print(exportPottery['identifier'])
exportPottery['sampleType']='ceramics'
print(exportPottery)


       identifier                                          relations  \
279    WESid_3400  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
884    WESid_3401  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
928    WESid_3370  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
963    WESid_3188  {'isRecordedIn': ['621f1722-39df-4acc-be7e-cdd...   
978    WESid_3283  {'isRecordedIn': ['5eb36fd9-b669-4358-9c04-a11...   
1161   WESid_3265  {'isRecordedIn': ['dba26d75-9e86-4fe7-b7f0-48d...   
1671   WESid_3263  {'isRecordedIn': ['dba26d75-9e86-4fe7-b7f0-48d...   
1701   WESid_3295  {'isRecordedIn': ['1f60bf87-7a09-4545-b36e-9bb...   
2310   WESid_3381  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
2380   WESid_3236  {'isRecordedIn': ['3d166975-b229-46bc-8038-1e2...   
2464   WESid_3274  {'isRecordedIn': ['5eb36fd9-b669-4358-9c04-a11...   
2593   WESid_3270  {'isRecordedIn': ['dba26d75-9e86-4fe7-b7f0-48d...   
3273   WESid_3212  {'isRecordedIn': ['621f1722-39df-4acc-be7e-cd

In [16]:
mudbrick= pdfield.filterDFBySubstringInField(samples, 'description', 'mud')
print(mudbrick)

       identifier                                          relations  \
4768   WESid_3348  {'isRecordedIn': ['f30a6f14-cb38-4977-b4f9-a47...   
21083  WESid_3345  {'isRecordedIn': ['f30a6f14-cb38-4977-b4f9-a47...   

                                         id geometry    type  height  \
4768   2d67c54a-5744-4dd7-bec1-16be1cede244      NaN  Sample     NaN   
21083  c8859d5f-96e7-4741-8db9-32b754b0e596      NaN  Sample     NaN   

      literature originalFilename shortDescription  width  ... institution  \
4768         NaN              NaN              NaN    NaN  ...         NaN   
21083        NaN              NaN              NaN    NaN  ...         NaN   

      projectSupervisor campaigns  epsgId  \
4768                NaN       NaN     NaN   
21083               NaN       NaN     NaN   

                                                 created  \
4768   {'user': 'mhaibt', 'date': '2023-03-24T11:16:2...   
21083  {'user': 'DAI-OA-185', 'date': '2023-03-15T09:...   

              

In [17]:
%reload_ext autoreload
%autoreload 2

samples = targetDF[targetDF['type']=='Sample']
samples = samples[~samples['identifier'].isin(exportPottery['identifier'])]
samples =  pd.concat([samples ,exportPottery], ignore_index=True)
print('ceramics: ' + str(len(samples[samples['sampleType']=='ceramics'])))
samplesTypeLumi= pdfield.filterDFBySubstringInField(samples, 'description', 'optical')
print('Lumi: ' + str(len(samplesTypeLumi)))
samplesTypeLumi['sampleType']='optical stimulated luminescence dating sample'
samples.update(samplesTypeLumi)



samplesTypeBodenprobe= pdfield.filterDFBySubstringInField(samples, 'sampleType', 'Bodenprobe')
samplesTypeBodenprobe['sampleType']='sedimentological sample'
samples.update(samplesTypeBodenprobe)

samplesTypePollen= pdfield.filterDFBySubstringInField(samples, 'sampleType', 'Pollenanalyse')
print('Pollen: ' + str(len(samplesTypePollen)))
samplesTypePollen['sampleType']='pollen sample'
samples.update(samplesTypePollen)

samplesTypeC14= pdfield.filterDFBySubstringInField(samples, 'sampleType', 'C14-Analyse')
print('C14: ' + str(len(samplesTypeC14)))
print(samplesTypeC14)
samplesTypeC14['sampleType']='radiocarbon dating sample'
samples.update(samplesTypeC14)

samplesTypecarbon= pdfield.filterDFBySubstringInField(samples, 'description', 'radiocarbon')
samplesTypecarbon['sampleType']='radiocarbon dating sample'

samples.update(samplesTypecarbon)
samplesupdated = samples
sampleTypenull = samplesupdated[samplesupdated['sampleType'].isnull()]
print(sampleTypenull)
samplesTypeSedimentol= pdfield.filterDFBySubstringInField(sampleTypenull, 'description', 'sedimentol')
print('Sedimentol: ' + str(len(samplesTypeSedimentol)))
samplesTypeSedimentol['sampleType']='sedimentological sample'
samplesupdated.update(samplesTypeSedimentol)

#sampleTypenull = samplesupdated[samplesupdated['sampleType'].isnull()]
print()
#sampleTypenull['sampleType']='sedimentological sample'
#samplesupdated.update(sampleTypenull)
#samples['context']=targetDF[targetDF['id']==borelocationid]['identifier']
#print(samples['context'])
#samples.apply(pdfield.getPathFromIsdepictedIn, imagestore=config['imagestore'], axis=1)

#mod_samples = samples[['identifier','type','relations']].reset_index()
#samples.drop(columns='index',inplace=True)

ceramics: 39
Lumi: 3
Pollen: 3
C14: 16
     identifier                                          relations  \
8    WESid_3381  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
18   WESid_3244  {'isRecordedIn': ['3d166975-b229-46bc-8038-1e2...   
39   WESid_3350  {'isRecordedIn': ['f30a6f14-cb38-4977-b4f9-a47...   
42   WESid_3227  {'isRecordedIn': ['621f1722-39df-4acc-be7e-cdd...   
51   WESid_3389  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
67   WESid_3383  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
73   WESid_3344  {'isRecordedIn': ['f30a6f14-cb38-4977-b4f9-a47...   
76   WESid_3201  {'isRecordedIn': ['621f1722-39df-4acc-be7e-cdd...   
80   WESid_3221  {'isRecordedIn': ['b22e7bbf-c650-4536-a337-64c...   
83   WESid_3341  {'isRecordedIn': ['f30a6f14-cb38-4977-b4f9-a47...   
85   WESid_3387  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
96   WESid_3220  {'isRecordedIn': ['b22e7bbf-c650-4536-a337-64c...   
111  WESid_3312  {'isRecordedIn': ['01dda821-c573-4

In [18]:

samples_mod = samplesupdated.apply(pdfield.getIdentifiersOfRelatedResources, relation='isRecordedIn', alldb=targetDF, outputfield='operation', axis=1)
samples_mod = samples_mod.apply(pdfield.getIdentifiersOfRelatedResources, relation='liesWithin', alldb=targetDF, outputfield='layer', axis=1)

samples_mod =samples_mod.apply(pdfield.clean_list_column, inputfield = 'operation', axis=1)
samples_mod =samples_mod.apply(pdfield.clean_list_column, inputfield = 'layer', axis=1)

samples_mod['context']= 'Drilling core: ' + samples_mod['operation'].astype(str)  + ' at depth: ' + samples_mod['spatialLocation'].astype(str) + ','  + 'sample of layer ' + samples_mod['layer'].astype(str) +'.'
print(samples_mod[samples_mod['sampleType']=='radiocarbon dating sample'])

     identifier                                          relations  \
8    WESid_3381  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
18   WESid_3244  {'isRecordedIn': ['3d166975-b229-46bc-8038-1e2...   
39   WESid_3350  {'isRecordedIn': ['f30a6f14-cb38-4977-b4f9-a47...   
42   WESid_3227  {'isRecordedIn': ['621f1722-39df-4acc-be7e-cdd...   
51   WESid_3389  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
67   WESid_3383  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
73   WESid_3344  {'isRecordedIn': ['f30a6f14-cb38-4977-b4f9-a47...   
76   WESid_3201  {'isRecordedIn': ['621f1722-39df-4acc-be7e-cdd...   
80   WESid_3221  {'isRecordedIn': ['b22e7bbf-c650-4536-a337-64c...   
83   WESid_3341  {'isRecordedIn': ['f30a6f14-cb38-4977-b4f9-a47...   
85   WESid_3387  {'isRecordedIn': ['73ba6474-69f7-4201-a59d-2e7...   
96   WESid_3220  {'isRecordedIn': ['b22e7bbf-c650-4536-a337-64c...   
111  WESid_3312  {'isRecordedIn': ['01dda821-c573-4168-99a5-926...   
133  WESid_3241  {'i

In [19]:
data=pd.read_csv("D:/UrukLarge/WES_paleoenvi/Proben_Bilder7/probenbilder7_list189.txt", sep="\n", header=None, names=['WESid'])
print(data.sort_values(['WESid'],ascending = [True]).reset_index())
print(samples_mod.sort_values(['identifier'],ascending = [True]).reset_index())
#print(data[~data['WESid'].isin(samples_mod['identifier'])])
#print(samples_mod[~samples_mod['identifier'].isin(data['WESid'])])

     index       WESid
0       45  WESid_3187
1       46  WESid_3188
2       47  WESid_3189
3       48  WESid_3190
4       49  WESid_3191
5       50  WESid_3192
6       51  WESid_3193
7       52  WESid_3194
8       53  WESid_3195
9       54  WESid_3196
10      55  WESid_3197
11      56  WESid_3198
12      57  WESid_3199
13      58  WESid_3200
14      59  WESid_3201
15      60  WESid_3202
16      61  WESid_3203
17      62  WESid_3204
18      63  WESid_3205
19      64  WESid_3206
20      65  WESid_3207
21      66  WESid_3208
22      67  WESid_3209
23      68  WESid_3210
24      69  WESid_3211
25      70  WESid_3212
26      71  WESid_3213
27      72  WESid_3215
28      73  WESid_3220
29      74  WESid_3221
30      75  WESid_3223
31      76  WESid_3226
32      77  WESid_3227
33      78  WESid_3228
34      79  WESid_3229
35      80  WESid_3230
36      81  WESid_3231
37      82  WESid_3232
38      83  WESid_3233
39      84  WESid_3234
40      85  WESid_3235
41      86  WESid_3236
42      87 

In [20]:
%reload_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', None)
samples_mod = samples_mod[['identifier','sampleType','context', 'type', 'relations']]
samples_mod = samples_mod.sort_values(['sampleType', 'identifier'],ascending = [True, True]).reset_index()
print(samples_mod)
#print('Total number of samples: ', str(len(samples_mod)))
pdfield.createPDF(samples_mod, imagestorepath=Path(config['imagestore'])/ config['db_name'], outputpath='D:/UrukLarge/WES_paleoenvi/Proben_Bilder/exportlist2023-1v2.pdf')

     index  identifier                                     sampleType  \
0      153  WESid_3191                                       ceramics   
1      156  WESid_3194                                       ceramics   
2      180  WESid_3195                                       ceramics   
3      174  WESid_3196                                       ceramics   
4      159  WESid_3198                                       ceramics   
5      158  WESid_3199                                       ceramics   
6      186  WESid_3202                                       ceramics   
7      166  WESid_3203                                       ceramics   
8      164  WESid_3208                                       ceramics   
9      169  WESid_3234                                       ceramics   
10     167  WESid_3235                                       ceramics   
11     176  WESid_3237                                       ceramics   
12     154  WESid_3238                             